# Les variables binaires en optimisation 
Tentative de clarification et étude par estelle derrien

- Valeurs : Elles prennent la valeur 0 ou 1 (non ou oui).
- Intérêt : Sélectionner une variable de décision , pénaliser une variable de décision, sélectionner une contrainte, discriminer une contrainte.
- Utilisation : Directement dans la fonction objectif, ou avec les contraintes.
- Déclaration : Elle diffère selon les solveurs, elle est plus facile dans certains.
- Semblable à : IF ( Si en Anglais) THEN , Semblable à un principe ON/OFF pour les machines. Equivaut à un 'trigger' en Anglais, un déclencheur.
- Relaté à : Les méthodes <b>"Big M"</b>.
- Permettent souvent de : Eviter de faire une optimisation non linéaire.
- Permettent d'attribuer une <b>pénalité</b> à une variable de décision, si sa valeur dépasse un certain seuil.
- On évite d'utiliser la fonctionnalité de limites (Bounds) sur les variables de décision souvent fournies avec le solveur, <b>en même temps que</b> les variables binaires, c'est soit l'un,soit l'autre.

# Exemples simples:

Si la production de x dépasse 20, alors la contrainte  N de coût maintenance s'applique, sinon, elle ne s'applique pas.

Si un objet X1(variable de décision) dépasse 20kgs, alors le container Z n'est pas utilisé, c'est le container Y qui est utilisé.

Si un élément chimique X1 (variable de décision) dépasse un seuil de 20grammes, alors, une pénalité lui est appliquée, ce qui privilégie l'élement chimique X2 dans l'optimisation finale.

Déclencher une machine X2 si la production sur la machine X1 est > 1000, sinon, ne pas la déclencher (Mode ON/OFF)



## 1/ Si la production de x dépasse 20, alors la contrainte  N de coût maintenance s'applique, sinon, elle ne s'applique pas.

In [2]:
# Importer la librairie Pulp sous le pseudo p
import pulp as p 
  
# Créer un programme linéaire de maximisation
Lp_prob = p.LpProblem('Problem', p.LpMaximize)  

# On spécifie le cout de maintenance
cout_maintenance = 10
  
# On Crée les variables du problème , x et y sont des objets que l'usine produit
x = p.LpVariable("x", lowBound = 0)   # Create a variable x >= 0 
y = p.LpVariable("y", lowBound = 0)   # Create a variable y >= 0 

# Comme on a besoin d'appliquer un cout de maintenance conditionnel, on a besoin 
# de définir une variable binaire qui va se déclencher si la production de x est supérieure
# à 20 objets !

z = 
  
# Ecrire la fonction objectif à maximizer qui nous donne un résultat en Euros 
Lp_prob +=  10.5 * x + 8.5 * y + 18.5 * z
  
# Les contraintes : 

# Heures de travail au mois , plus il est grand, plus le prfoit augmente, moins le temps de travail sur un objet est long, plus on en produit et le profit augmente 
Lp_prob += 3 * x + 3 * y + 2 * z <= 420

# Il faut produire au minimum ce nombre d'éléments  :
Lp_prob += x  >= 100
Lp_prob += y  >= 10
Lp_prob += z  >= 10
  
# Afficher le problème linéaire
print(Lp_prob) 


NameError: name 'z' is not defined